In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('../../'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import numpy as np
import matplotlib.pyplot as plt

from src.extra.model import ModelDistribution
from src.extra.pprint import print_model
from src.core.core import reset_model

from src.greedy.greedy import greedy_algorithm
from src.greedy.resource_allocation_policy import SumPercentage
from src.greedy.server_selection_policy import SumResources
from src.greedy.task_prioritisation import UtilityDeadlinePerResource

from src.optimal.fixed_optimal import fixed_optimal
from src.core.fixed_task import FixedTask, SumSpeedPowFixedPrioritisation

alibaba_model = ModelDistribution('../realistic.mdl', num_tasks=100, num_servers=10)

In [3]:
def print_results(result, tasks, servers):
    print(f'Social welfare: {result.social_welfare:.3f}, Percentage tasks allocated: {result.percentage_tasks_allocated:.3f}\n')
    for server in servers:
        storage_usage = sum(task.required_storage for task in server.allocated_tasks) / server.storage_capacity
        computational_usage = sum(task.compute_speed for task in server.allocated_tasks) / server.computation_capacity
        bandwidth_usage = sum(task.loading_speed + task.sending_speed for task in server.allocated_tasks) / server.bandwidth_capacity
        print(f'{server.name} resouce usage - Stor: {storage_usage:.3f}, Comp: {computational_usage:.3f}, Band: {bandwidth_usage:.3f}')
    
    avg_loading_speed = np.mean([task.loading_speed for task in tasks if task.running_server])
    avg_compute_speed = np.mean([task.compute_speed for task in tasks if task.running_server])
    avg_sending_speed = np.mean([task.sending_speed for task in tasks if task.running_server])
    print(f'\nAvg loading speed: {avg_loading_speed}, compute speed: {avg_compute_speed}, sending speed: {avg_sending_speed}')
    avg_loading_time = np.mean([task.required_storage / task.loading_speed for task in tasks if task.running_server])
    avg_compute_time = np.mean([task.required_computation / task.compute_speed for task in tasks if task.running_server])
    avg_sending_time = np.mean([task.required_results_data / task.sending_speed for task in tasks if task.running_server])
    print(f'Avg loading time: {avg_loading_time}, compute time: {avg_compute_time}, sending time: {avg_sending_time}')

In [4]:
alibaba_model.storage_scaling = 100
alibaba_model.computational_scaling = 0.25
alibaba_model.results_data_scaling = 100

In [5]:
tasks, servers = alibaba_model.generate()

In [6]:
result = greedy_algorithm(tasks, servers, UtilityDeadlinePerResource(), SumResources(), SumPercentage())
print_results(result, tasks, servers)

Social welfare: 756.980, Percentage tasks allocated: 0.970

medium 0 resouce usage - Stor: 0.980, Comp: 0.010, Band: 0.180
large 1 resouce usage - Stor: 1.000, Comp: 0.011, Band: 0.167
xlarge 2 resouce usage - Stor: 0.990, Comp: 0.036, Band: 0.130
large 3 resouce usage - Stor: 0.987, Comp: 0.015, Band: 0.180
large 4 resouce usage - Stor: 1.000, Comp: 0.050, Band: 0.287
xlarge 5 resouce usage - Stor: 0.990, Comp: 0.091, Band: 0.250
xlarge 6 resouce usage - Stor: 0.990, Comp: 0.091, Band: 0.320
large 7 resouce usage - Stor: 1.000, Comp: 0.027, Band: 0.173
2xlarge 8 resouce usage - Stor: 0.976, Comp: 0.081, Band: 0.356
large 9 resouce usage - Stor: 0.987, Comp: 0.031, Band: 0.193

Avg loading speed: 1.5257731958762886, compute speed: 12.948453608247423, sending speed: 2.5670103092783507
Avg loading time: 6.632547864506628, compute time: 21.156490446898314, sending time: 18.675733433738284


In [7]:
reset_model(tasks, servers)
fixed_tasks = [FixedTask(task, SumSpeedPowFixedPrioritisation()) for task in tasks]
foreknowledge_fixed_tasks = [FixedTask(task, SumSpeedPowFixedPrioritisation(), resource_foreknowledge=True) 
                             for task in tasks]

In [8]:
reset_model(tasks, servers)
result = fixed_optimal(fixed_tasks, servers, time_limit=4)
print_results(result, fixed_tasks, servers)

Social welfare: 361.060, Percentage tasks allocated: 0.260

medium 0 resouce usage - Stor: 0.800, Comp: 0.588, Band: 0.880
large 1 resouce usage - Stor: 0.800, Comp: 0.350, Band: 1.000
xlarge 2 resouce usage - Stor: 0.690, Comp: 0.942, Band: 1.000
large 3 resouce usage - Stor: 0.867, Comp: 0.662, Band: 1.000
large 4 resouce usage - Stor: 0.800, Comp: 0.375, Band: 0.967
xlarge 5 resouce usage - Stor: 0.990, Comp: 0.558, Band: 0.995
xlarge 6 resouce usage - Stor: 0.300, Comp: 0.163, Band: 0.925
large 7 resouce usage - Stor: 0.867, Comp: 0.237, Band: 0.940
2xlarge 8 resouce usage - Stor: 0.952, Comp: 0.184, Band: 0.984
large 9 resouce usage - Stor: 0.667, Comp: 0.312, Band: 0.973

Avg loading speed: 57.88461538461539, compute speed: 391.8269230769231, sending speed: 5.576923076923077
Avg loading time: 0.5378205128205128, compute time: 0.03716700103984153, sending time: 8.323568205298974


In [9]:
reset_model(tasks, servers)
result = fixed_optimal(foreknowledge_fixed_tasks, servers, time_limit=10)
print_results(result, foreknowledge_fixed_tasks, servers)

Social welfare: 758.170, Percentage tasks allocated: 0.990

medium 0 resouce usage - Stor: 1.000, Comp: 0.014, Band: 0.240
large 1 resouce usage - Stor: 1.000, Comp: 0.029, Band: 0.387
xlarge 2 resouce usage - Stor: 1.000, Comp: 0.015, Band: 0.285
large 3 resouce usage - Stor: 1.000, Comp: 0.021, Band: 0.440
large 4 resouce usage - Stor: 1.000, Comp: 0.017, Band: 0.273
xlarge 5 resouce usage - Stor: 1.000, Comp: 0.020, Band: 0.335
xlarge 6 resouce usage - Stor: 1.000, Comp: 0.022, Band: 0.515
large 7 resouce usage - Stor: 1.000, Comp: 0.050, Band: 0.720
2xlarge 8 resouce usage - Stor: 1.000, Comp: 0.027, Band: 0.504
large 9 resouce usage - Stor: 1.000, Comp: 0.034, Band: 0.593

Avg loading speed: 3.1717171717171717, compute speed: 5.9393939393939394, sending speed: 4.292929292929293
Avg loading time: 4.4204790563302145, compute time: 27.86787455577056, sending time: 13.313455259667379
